In [1]:
import pandas as pd
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [80]:
import matplotlib.pyplot as plt
import seaborn as sns
from autorocks.viz import plots_setup
import torch
import gpytorch
import botorch.posteriors
from botorch import models
from botorch.optim import fit
import torch
import numpy as np
import pandas as pd
import botorch

output_location = "/Users/salabed/workspace/latex_writings/thesis/phd_dissertation/Chapters/Background/Figures/"

tkwargs = {
    "dtype": torch.double,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
}
problem = botorch.test_functions.BraninCurrin(negate = True, noise_std = 0).to(**tkwargs)

ref_point = problem.ref_point
standard_bounds = torch.zeros(2, problem.dim, dtype = torch.float64)
standard_bounds[1] = 1

import warnings
from botorch.exceptions import BadInitialCandidatesWarning

warnings.filterwarnings("ignore", category = BadInitialCandidatesWarning)
# warnings.filterwarnings("ignore", category=RuntimeWarning)


In [81]:
num_observations = 10
num_restarts = 3
num_optimization_rounds = 100
MC_SAMPLES = 128

initial_train_x, initial_train_y = plots_setup.generate_data(num_observations, problem)

# test_x, test_y = plots_setup.generate_data(100, problem)

In [53]:
from botorch.sampling import SobolQMCNormalSampler
from botorch.utils.multi_objective.box_decompositions import (DominatedPartitioning, FastNondominatedPartitioning)
# Train two independent GP to model the tasks. 

from botorch.models.transforms.outcome import Standardize
import botorch.acquisition.multi_objective

independent_pred = []
for restart in range(num_restarts):
    train_x = initial_train_x.clone().detach()
    print(f"{restart=}")
    for step in range(num_optimization_rounds):
        # fit the model 
        train_y = problem(train_x)
        model = models.SingleTaskGP(
            train_X = train_x,
            train_Y = train_y,
            outcome_transform = Standardize(m = 2)
        )
        # Fit and train the model
        mll = gpytorch.mlls.ExactMarginalLogLikelihood(model.likelihood, model)
        fit.fit_gpytorch_mll_scipy(mll)

        model.eval()

        with torch.no_grad():
            pred = model.posterior(botorch.utils.transforms.normalize(train_x, problem.bounds)).mean
        partitioning = FastNondominatedPartitioning(
            ref_point = problem.ref_point,
            Y = pred,
        )

        acf = botorch.acquisition.multi_objective.qExpectedHypervolumeImprovement(model,
                                                                                  ref_point = problem._ref_point,
                                                                                  partitioning = partitioning,
                                                                                  sampler = SobolQMCNormalSampler(
                                                                                      sample_shape = torch.Size(
                                                                                          [MC_SAMPLES])))
        new_candidate, _ = botorch.optim.optimize_acqf(
            acf, bounds = standard_bounds, q = 1, num_restarts = 1, raw_samples = 64)

        new_candidate = botorch.utils.transforms.unnormalize(new_candidate, problem.bounds).detach()
        train_x = torch.concat([train_x, new_candidate])
        new_y = problem(new_candidate)

        best_f = train_y.sum(-1, keepdim = True).max()

        bd = DominatedPartitioning(ref_point = problem.ref_point, Y = train_y)
        # volume = partitioning.compute_hypervolume().item()
        volume = bd.compute_hypervolume().item()
        independent_pred.append({
            "score": float(volume),
            "branin": float(new_y[:, 0].cpu().detach().numpy()),
            "currin": float(new_y[:, 1].cpu().detach().numpy()),
            "candidate": new_candidate.detach().cpu().numpy(),
            "best_f": float(best_f.detach().cpu().numpy()),
            "model": "Independent",
            "step": step,
            "restart": restart,
        })

In [103]:
from botorch.utils.multi_objective.box_decompositions import FastNondominatedPartitioning
# Train two independent GP to model the tasks. 

from botorch.models.transforms.outcome import Standardize
import botorch.acquisition.multi_objective

mt_pred = []

for restart in range(num_restarts):
    train_x = initial_train_x.clone()
    print(f"{restart=}")
    for step in range(num_optimization_rounds):
        # fit the model 
        train_y = problem(train_x)
        train_x_f1 = torch.hstack([train_x, torch.zeros(train_x.shape[0], 1)])
        train_x_f2 = torch.hstack([train_x, torch.ones(train_x.shape[0], 1)])

        model = models.FixedNoiseMultiTaskGP(
            train_X = torch.concat([train_x_f1, train_x_f2]),
            train_Y = train_y.T.reshape(-1, 1),
            train_Yvar = torch.zeros_like(train_y.T.reshape(-1, 1)),
            task_feature = -1,
            # outcome_transform = Standardize(m = 1),
        )
        # Fit and train the model
        mll = gpytorch.mlls.ExactMarginalLogLikelihood(model.likelihood, model)
        fit.fit_gpytorch_mll_scipy(mll)

        model.eval()

        with torch.no_grad():
            pred = model.posterior(train_x).mean.reshape(-1, 2)
        partitioning = FastNondominatedPartitioning(
            ref_point = problem.ref_point,
            Y = pred,
        )

        acf = botorch.acquisition.multi_objective.ExpectedHypervolumeImprovement(model,
                                                                                 ref_point = problem.ref_point,
                                                                                 partitioning = partitioning,
                                                                                 sampler = SobolQMCNormalSampler(
                                                                                     sample_shape = torch.Size(
                                                                                         [MC_SAMPLES])))
        new_candidate, _ = botorch.optim.optimize_acqf(
            acf, bounds = problem.bounds, q = 1, num_restarts = 4, raw_samples = 512)

        new_candidate = botorch.utils.transforms.unnormalize(new_candidate, problem.bounds).detach()
        train_x = torch.concat([train_x, new_candidate])
        new_y = problem(new_candidate)

        best_f = train_y.sum(-1, keepdim = True).max()
        bd = DominatedPartitioning(ref_point = problem.ref_point, Y = train_y)
        volume = bd.compute_hypervolume().item()
        # volume = partitioning.compute_hypervolume().item()
        mt_pred.append({
            "score": float(volume),
            "branin": float(new_y[:, 0].cpu().detach().numpy()),
            "currin": float(new_y[:, 1].cpu().detach().numpy()),
            "candidate": new_candidate.detach().cpu().numpy(),
            "best_f": float(best_f.detach().cpu().numpy()),
            "model": "MultiTask",
            "step": step,
            "restart": restart,
        })

In [197]:

all_pred = independent_pred + mt_pred

df = pd.DataFrame(all_pred)
df['score'] = df['score'].astype(float)
df['log_hv'] = np.log10(problem.max_hv - df['score'])
df['step'] = df['step'].astype(int)
df['restart'] = df['restart'].astype(int)
df['model'] = df['model'].str.replace('Independent', 'SingleTask')
df

In [198]:
plt.style.use("ggplot")
sns.set_theme(style = "white", rc = {"axes.spines.right": False, "axes.spines.top": False})
sns.set_context("paper")  # , font_scale=1.5, rc={"lines.linewidth": 1.5})
plt.rcParams["svg.fonttype"] = "none"
plt.rcParams["font.family"] = "Arial"
plt.rc("text", usetex = False)
plt.rc("xtick", labelsize = "large")
plt.rc("ytick", labelsize = "large")
plt.rc("axes", labelsize = "large")
plt.rc("pdf", use14corefonts = True)

# Initialize plot
f, ax = plt.subplots(1, 1, figsize = (4, 3))

# x_axis is the number of observation
# y_axis is the score 
df
sns.lineplot(df, x = "step", y = "log_hv", hue = "model", ax = ax)
ax.set(xlabel = "Step", ylabel = "Log Hypervolume Difference")

# ax.set(ylim = (-1, 5))
# # Plot training data as red stars
# ax.plot(train_x_full.numpy(), train_y_full.numpy(), 'r*')
# # Plot predictive means as blue line
# ax.plot(test_x.numpy(), mean.numpy(), 'b')
# # Shade between the lower and upper confidence bounds
# ax.fill_between(test_x.numpy(), lower.squeeze().numpy(), upper.squeeze().numpy(), alpha = 0.2)
# ax.legend(['Observed Data', "Real Function", 'Mean', 'Confidence'])
# plt.gca().xaxis.set_major_locator(plt.NullLocator())
# plt.gca().yaxis.set_major_locator(plt.NullLocator())

output_format = "pdf"
f.savefig(f"{output_location}/mt_score_comparison.{output_format}", bbox_inches = "tight", format = f"{output_format}",
          dpi = 300)


In [208]:
plt.style.use("ggplot")
sns.set_theme(style = "white", rc = {"axes.spines.right": False, "axes.spines.top": False})
sns.set_context("paper")  # , font_scale=1.5, rc={"lines.linewidth": 1.5})
plt.rcParams["svg.fonttype"] = "none"
plt.rcParams["font.family"] = "Arial"
plt.rc("text", usetex = False)
plt.rc("xtick", labelsize = "large")
plt.rc("ytick", labelsize = "large")
plt.rc("axes", labelsize = "large")
plt.rc("pdf", use14corefonts = True)

# Initialize plot
# f, ax = plt.subplots(1, 2, figsize = (16, 9))

# x_axis is the number of observation
# y_axis is the score 
df["Pareto-Front"] = botorch.utils.multi_objective.is_non_dominated(torch.tensor(df[['branin', 'currin']].to_numpy()))
grid = sns.FacetGrid(df, col = "model", palette = "tab20c", col_wrap = 2, height = 4)
grid.map(sns.scatterplot, "branin", "currin", 'step', "step", "Pareto-Front")
grid.set(xlabel = "Branin", ylabel = "Currin")
grid.add_legend()

# sns.scatterplot(df[df['model'] == 'Independent'], x = "branin", y = "currin",  ax = ax[0], hue = "step")
# sns.scatterplot(df[df['model'] == 'MultiTask'], x = "branin", y = "currin", ax = ax[1], hue = "step")
# 

# ax.set(ylim = (-1, 5))
# # Plot training data as red stars
# ax.plot(train_x_full.numpy(), train_y_full.numpy(), 'r*')
# # Plot predictive means as blue line
# ax.plot(test_x.numpy(), mean.numpy(), 'b')
# # Shade between the lower and upper confidence bounds
# ax.fill_between(test_x.numpy(), lower.squeeze().numpy(), upper.squeeze().numpy(), alpha = 0.2)
# ax.legend(['Observed Data', "Real Function", 'Mean', 'Confidence'])
# plt.gca().xaxis.set_major_locator(plt.NullLocator())
# plt.gca().yaxis.set_major_locator(plt.NullLocator())

plt.rcParams['axes.unicode_minus'] = False
output_format = "pdf"
grid.fig.savefig(f"{output_location}/mt_mobo.{output_format}", bbox_inches = "tight", format = f"{output_format}", 
                 dpi = 300)


In [193]:
df.to_csv('./mt_bo.csv')